In [28]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick
#OPTION 1: use the formula P = \int_G \rho(g)\rho(g)^* dg, and take a square root to find Q s.t. P = Q^2
#OPTION 2: define a new invariant inner product compute an orthonormal basis w.r.t. 
#this inner product by computing the Gram matrix A and using A.gram_schmidt()

In [70]:
#find the change-of-basis matrix M for which A = M*GS where GS is the Gram-Schmidt orthonormal basis of A
def unitary_change_of_basis(SGA,partition):
    rho = SGA.specht_module(partition).representation_matrix
    group_size = SGA.group().cardinality()
    P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
    return P.principal_square_root()

In [91]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition,SGA,unitary=False):
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    d_part = specht_module.dimension()
    group_size = SGA.group().cardinality()
    if unitary:
        Q = unitary_change_of_basis(SGA,partition)
        return sqrt(d_part/group_size)*sum(f(g)*Q.inverse()*rho(g)*Q for g in SGA.group())
    else:
        return sum(f(g)*rho(g) for g in SGA.group())

In [34]:
#define the delta function delta_s(t) = {1 if s == t, 0 otherwise}
delta = lambda s: lambda t: 1 if t == s else 0

In [57]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
from sage.misc.flatten import flatten
def unitary_dft(SGA):
    return matrix([flatten([hat(delta(g),partition,SGA,unitary=True).list() for partition in Partitions(SGA.group().degree())]) for g in G]).transpose()

In [ ]:
#is the SGA DFT unitary?

In [44]:
SGA = SymmetricGroupAlgebra(QQ,3)

In [45]:
A = SGA.dft(); A

[   1    1    1    1    1    1]
[   1  1/2   -1 -1/2 -1/2  1/2]
[   0  3/4    0  3/4 -3/4 -3/4]
[   0    1    0   -1    1   -1]
[   1 -1/2    1 -1/2 -1/2 -1/2]
[   1   -1   -1    1    1   -1]

In [47]:
#check if A*A^T == Id. it's not, but the columns are orthonormal
A*A.transpose()

[  6   0   0   0   0   0]
[  0   3   0   0   0   0]
[  0   0 9/4   0   0   0]
[  0   0   0   4   0   0]
[  0   0   0   0   3   0]
[  0   0   0   0   0   6]

In [60]:
SGA.group().cardinality()

6

In [93]:
unitary_dft(SGA).simplify_full()

[ 1/6*sqrt(6)  1/6*sqrt(6)  1/6*sqrt(6)  1/6*sqrt(6)  1/6*sqrt(6)  1/6*sqrt(6)]
[ 1/3*sqrt(3)            0          1/2 -1/6*sqrt(3) -1/6*sqrt(3)         -1/2]
[           0  1/3*sqrt(3) -1/6*sqrt(3)          1/2         -1/2 -1/6*sqrt(3)]
[           0  1/3*sqrt(3) -1/6*sqrt(3)         -1/2          1/2 -1/6*sqrt(3)]
[ 1/3*sqrt(3)            0         -1/2 -1/6*sqrt(3) -1/6*sqrt(3)          1/2]
[ 1/6*sqrt(6) -1/6*sqrt(6) -1/6*sqrt(6)  1/6*sqrt(6)  1/6*sqrt(6) -1/6*sqrt(6)]

In [ ]:
#check that the DFT is unitary
(unitary_dft(3)*unitary_dft(3).transpose()).simplify_full() == identity_matrix(factorial(3))

In [ ]:
eigs = matrix(CDF,unitary_dft(3).simplify_full()).eigenvalues()

In [ ]:
eigs

In [ ]:
#what are the eigenvalues?
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [ ]:
eigs = A.eigenvalues(); eigs

In [ ]:
[abs(eig) for eig in eigs]

In [ ]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((A*A.transpose()).numpy().diagonal()))